In [139]:
#data should be cleaned and it should have null values or any values that are outside the given range and values that do not make sense 
#first step data loading 
#dataframe is 2D labelled data structure 
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os

In [140]:
nltk.download('vader_lexicon')

[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [141]:
app_df=pd.read_csv('/Users/prateeksingh/Desktop/Play Store Data.csv')
rev_df=pd.read_csv('/Users/prateeksingh/Desktop/User Reviews.csv')

In [142]:
app_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [143]:
rev_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [144]:
#step2: Handling your missing values and duplicates 
#to handle the missing values first we have to know how to regnize the missing values
#there is function called df.isnull() this depicts the missing values in the dataframe 
#df.dropna()this func removes rowa rows aND columns that contain the missing values
#to replace null value with numerics value will use function df.fillna() with mean value or median value 
#how to handle and duplicatle data because it acn distort your analysis fuc is used id df.duplicated() if identifiy the rows and by func df.drop_duplicates() to remove rows

In [145]:
app_df=app_df.dropna(subset=['Rating']) #to remove rows having na in ratings columns 
#filling missing value with the most frequent value by taking mode of the column  
for column in app_df.columns:
    app_df[column].fillna(app_df[column].mode()[0],inplace=True)
#now droping the duplicates value
app_df.drop_duplicates(inplace=True)
#now i want to filter the data whether rating is less than or equal to 5 so that rating having above 5 should be ignored
app_df=app_df=app_df[app_df['Rating']<=5]
#now removing null value from reviews table from translated reviews colomn 
rev_df.dropna(subset=['Translated_Review'],inplace=True) #reviews are important for sentiment analysis 

/var/folders/mf/kt92kq0j5w9f44f6l_74f0c40000gn/T/ipykernel_95329/3114029962.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [146]:
#now transforming the data for our analysis by transforming the data types for our visualization function called astype() converts the column data type 
#convert the install column top numeric by removing , and +
app_df['Installs']=app_df['Installs'].str.replace(',','').str.replace('+','').astype(int)
#converting the price column to numeric by removing the $
app_df['Price']=app_df['Price'].str.replace('$','').astype(float)

In [147]:
app_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs            int64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [148]:
#we have to merge the dataset to get the detailed view of each app include attributes and user feedback merge can be done done by using the same column in both table here it is 'app'
new_df=pd.merge(app_df,rev_df,on='App',how='inner')

In [149]:
new_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000


In [150]:
#next step: data tranformation and EDA and how to uncover insight from your cleaned data
#how to transform the datatypes formats such as installs column particular data format eg ddmmyy into mmddyy 
#data transformation is the process of converting data from one format to another and to make it suitable for ML model and DL model 
#adjusting the values which are not in the range in the columns i known as normalizing the data 
#categorizing the data grouping the data into certain categories 
#1st step converting the data format
app_df['Reviews']=app_df['Reviews'].astype(int)
#changing format of app size in in one unit like in mb or gbs ot kbs
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M',''))
    elif 'K' in size:
        return float(size.replace('K',''))/1024
    else:
        return np.nan
app_df['Size']=app_df['Size'].apply(convert_size) 


In [151]:
app_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [152]:
#log transformed by taking logarithimic of the variable help managing skewed distribution or data which is normally distributed more suitable for statistical analysis
#doing logarithim change here 
app_df['Log_Installs']= np.log1p(app_df['Installs'])
app_df['Log_Reviews']=np.log1p(app_df['Reviews'])

In [153]:
#categorize the rating 
def rating_group(rating):
    if rating >= 4:
        return 'Top Rated App'
    elif rating >= 3:
        return 'Above Average'
    elif rating >= 2:
        return 'Average'
    else:
        return 'Below Average'
app_df['Rating_group']=app_df['Rating'].apply(rating_group)
#this will help to segment the apps based on user satisfaction why? it is because it helps me to get ceertain patterns in the data 

In [154]:
#deriving a metric called as a revenue coloumn 
app_df['Revenue']=app_df['Price']*app_df['Installs']
#revenue matters at the end sometime revenue increases but ratings are very bad 

In [155]:
app_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Log_Reviews,Rating_group,Revenue
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,9.210440,5.075174,Top Rated App,0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,13.122365,6.875232,Above Average,0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,15.424949,11.379520,Top Rated App,0.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,17.727534,12.281389,Top Rated App,0.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,11.512935,6.875232,Top Rated App,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up,6.216606,2.079442,Top Rated App,0.0
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,8.517393,3.663562,Top Rated App,0.0
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,4.615121,1.609438,Top Rated App,0.0
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device,6.908755,4.744932,Top Rated App,0.0


In [156]:
#sentiment analysis by NLP 
#NLP is branch under artificial intellegence so that computer can understand, interpret and generate the human language in a way which computer can understand and anlyze it
#it is combination of Computer science and linguistic 
#1st Objective:
#NLP help enable a machine to understand and read the content of the written or the spoken language and it also involves the breaking down of text into small components and
#analyze these components.
#there various part of NLP 1st is "tokenization" where we do is we split the text or sentence into words and phrases 2nd is "parsing" where i'm analyzing the grammer structure 
# analyzing what grammer is being used whether its past tense present tense participle and then comes the part of "speech tagging" that is POS tagging means what role that 
#each word plays in a sentence 
#another objective: is to generate the language by understanding the labnguage i'm trying to generate in natural language which is appropiate and meaningfull language like
#chat gpt like chat bot .... how gmail analyse wheate mail is a spam or not, the sentiment analysis the customer reviews the text classification, i wanr to convert text 
#from hindi to english  translation speech recognition like siri google assistant again NLP plays an major role
#"Stemming and lemmatization" basically changing it into the root form like i'm using running than convert it into run or embedding any words 
# Here we are using sentiment analysis which is domain under NLP technique where it is use to determine the sentiment suppose i'm writing an review and the monitoring whether
#my review is good or bad so checking it manually is going to take long time instead use an nlp model
#sentiment analysis classify into 3 forms 1.positive, 2.Negative 3.Neutral which helps in understanding the customer opinion market trend or public sentiments 
#why it is imp it crucial to analyze the rev iew eg. clg need a feedback form because they want to know want wents wrong so that they can make change, company wants to know
#customer is statisfied which depends there future decisions absed on these review some application of sentiment analysis are customer feedback 
#natural language toolkit(NLTK) library of python used for sentiment analysis it has a tool "sentiment intensity analyzer"

In [157]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()

In [158]:
#polarity scores SIA
#positive negative nuetral and compound -1 means very negative, +1 means very positive 

In [159]:
rev_df['Sentiment_Score']=rev_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])#it will add another column and going to add the polarity scores
#.apply(lambda x: ...)This applies a function to each row in the 'Translated_Review' column.
#str(x) Ensures that even if some reviews are missing (NaN values), they are converted to a string (to avoid errors).
#sia.polarity_scores(str(x)) Uses sia.polarity_scores() (VADER sentiment analysis) to analyze the sentiment of the review text.
#['compound'] Extracts the compound sentiment score, which is a single number that represents the overall sentiment:

In [160]:
rev_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Sentiment_Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,0.9531
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,0.6597
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,0.6249
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,0.6369
5,10 Best Foods for You,Best way,Positive,1.00,0.300000,0.6369


In [161]:
#now i want to extract the year or create a year column like i just want year only 
app_df['Last Updated']=pd.to_datetime(app_df['Last Updated'],errors='coerce')#convert column into date time format and remove the error
app_df['Year']=app_df['Last Updated'].dt.year#after converting extracting the year by dt.year function

In [162]:
app_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Log_Reviews,Rating_group,Revenue,Year
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,9.210440,5.075174,Top Rated App,0.0,2018
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,13.122365,6.875232,Above Average,0.0,2018
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,15.424949,11.379520,Top Rated App,0.0,2018
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,17.727534,12.281389,Top Rated App,0.0,2018
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,11.512935,6.875232,Top Rated App,0.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,2017-06-18,1.0.0,4.1 and up,6.216606,2.079442,Top Rated App,0.0,2017
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,2017-07-25,1.48,4.1 and up,8.517393,3.663562,Top Rated App,0.0,2017
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,2018-07-06,1.0,4.1 and up,4.615121,1.609438,Top Rated App,0.0,2018
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,2015-01-19,Varies with device,Varies with device,6.908755,4.744932,Top Rated App,0.0,2015


In [163]:
#plotly visualization lib allows use to create intretractive graphs supports very wide range of graphs it helps people to engage with real time data 
#difference between plotly and matplotly graph when hover on plotly graph you get more information etc after clicking it changes dynamically 
#why to save plotly graph saving plot as an HTML file  is very useful because it help me to share interactive visualiztion via web pages to show it to the client 
#fig.write_html("interactive_plot.html") to save the graph ot plots 
#difference between static and interactive visualization static is fixed images or plot are fixed used very traditional chart like histogram, pie chart it's very simple and easy to create
#disadvange limited information of the data 
#interactive visualizatio allows user to engage with the data by zooming, filter, hover over the element, even alter the visualization dynamically and maker to feel and explore
#the data it give lots of flexibility disadvantage it rquires advanced knopwledge 

In [164]:
import os
import os

# Path to Desktop folder
desktop_path = os.path.expanduser("~/Desktop/graph")

# Create folder if it doesn't exist
if not os.path.exists(desktop_path):
    os.makedirs(desktop_path)

print(f"Folder created at: {desktop_path}")

Folder created at: /Users/prateeksingh/Desktop/graph


In [165]:
#to mention what save_plot_as_html func is doing 
plot_containers = "" #defining plot container like box where plot is has to present

In [166]:
def save_plot_as_html(fig,filename,insight):
    global plot_containers
    filepath=os.path.join(desktop_path,filename)#creating the file 
    html_content=pio.to_html(fig,full_html=False,include_plotlyjs='inline')#merging the plotly into html file 
    #append the plot and it's insight to the plot_container  
    plot_containers += f"""
    <div class = "plot-containers" id="{filename}" onclick ="openPlot('{filename}')"> #combining it to the plot containers 
        <div class="plot">{html_content}</div> 
        <div class="insights">{insight}</div> 
    </div>
    """
    fig.write_html(filepath,full_html=False,include_plotlyjs='inline')

In [167]:
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size': 16}
axis_font={'size':12}

In [168]:
#fig 1
category_counts=app_df['Category'].value_counts().nlargest(10)#nlargest is the function where i'll get top 10app categories 
fig1=px.bar(
    x=category_counts.index,
    y=category_counts.values,
    labels={'x':'Category','y':'Count'},
    title='Top categories on playstore',
    color=category_counts.index,
    color_discrete_sequence=px.colors.sequential.Plasma,
    width=400,
    height=300
)
#code for layout of the fig1 bar graph 
fig1.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#everybody cannot understand the graph so in that scenireo we should explain in an one line 
#fig1.update_traces(marker=dict(marker=dict(line=dict(color='white',width=1))))
save_plot_as_html(fig1,"Category Graph 1.html","The top Categories on the playstore are dominated by tools, entertainment, and productivity apps")

In [169]:
#type analysis plot figure 2 objective analyzing distribution of free vs paid apps how many installs of the free and paid apps are there so i'm analyzing the distribution so i'll
#be using the pie chart here
type_counts=app_df['Type'].value_counts()
fig2=px.pie (  
    values=type_counts.values,
    names=type_counts.index, 
    title='App Type Distribution',
    color_discrete_sequence=px.colors.sequential.RdBu,
    width=400,
    height=300
)
#code for layout of the fig1 pie chart
fig2.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    margin=dict(l=10,r=10,t=30,b=10)
)
#everybody cannot understand the graph so in that scenireo we should explain in an one line 
save_plot_as_html(fig2,"Type Graph 2.html","Most apps on the Playstore are free, indicating a strategy users first and monetize through ads or in app purchase")

In [170]:
#Rating distribution Figure 3 objective is to examine how the app rating is distributed we do not need variable 
fig3=px.histogram(  
    app_df,
    x='Rating',
    nbins=20,
    title='Rating Distribution',
    color_discrete_sequence=['#636EFA '],
    width=400,
    height=300
)
#code for layout of the fig1 histogram graph 
fig3.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig3,"Rating Graph 3.html","Ratings are skewed towards higher values, suggesting that most apps are rated favorably by users")

In [171]:
#Figure 4 analyze a sentiment scores from app reviews we will use a bar chart 
sentiments_counts=rev_df['Sentiment_Score'].value_counts()
fig4=px.bar(
    x=sentiments_counts.index,
    y=sentiments_counts.values,
    labels={'x':'Sentiment_Score','y':'Count'},
    title='Sentiment Distribution', 
    color=sentiments_counts.index,
    color_discrete_sequence=px.colors.sequential.RdPu,
    width=400,
    height=300
)
#code for layout of the fig1 bar graph 
fig4.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig4,"Sentiment Graph 4.html","The sentiments in revies show a mix of positive and negative feedback, with a slight lean towards positive")

In [172]:
#visualizing which app category has more installs weather its gaming genre or entertainment etc 
#figure 5 using bar chart 
installs_by_category=app_df.groupby('Category')['Installs'].sum().nlargest(10)#combine the values of installs and then it is gropup on the basis of category
fig5=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    orientation='h',
    labels={'x':'Installs','y':'Category'},
    title='Installs by Category', 
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Blues,
    width=400,
    height=300
)
#code for layout of the fig1 bar graph 
fig5.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig5,"Installs Graph 5.html","The Categories with most installs are social and communication apps, reflecting there broad appeal and daily usage")#insight

In [173]:
#plotly is a powerfull lib use for creating interactive web based visualization
# what is the no. of app update over the year it means we will use line plot 
#figure6
updates_per_year=app_df['Last Updated'].dt.year.value_counts().sort_index()#extracting the year and doing value count of it 
fig6=px.line(
    x=updates_per_year.index,
    y=updates_per_year.values,
    labels={'x':'Years','y':'No. of Updates'},
    title='no. of update over the year', 
    color_discrete_sequence=['#AB63FA'],
    width=plot_width,
    height=plot_height
)
#code for layout of the fig1 bar graph 
fig6.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig6,"Updates Graph 6.html","Updates have been increasing over the years, showing that developer are actively maintaing and maintaing their Apps")

In [174]:
#comparing the revenue that is generated by the app category by using bar plot distribution across the top 10 categories 
 #fig 7
revenue_by_category=app_df.groupby('Category')['Installs'].sum().nlargest(10)#combine the values of installs and then it is gropup on the basis of category
fig7=px.bar(
    x=revenue_by_category.index,
    y=revenue_by_category.values,
    labels={'x':'Category','y':'Revenue'},
    title='revenue by Category', 
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Greens,
    width=400,
    height=300
)
#code for layout of the fig1 bar graph 
fig7.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig7,"Revenue Graph 7.html","Categories such as business and Productivity lead in revenue genration, indicating there monetization potential")

In [175]:
#let's count genre and create a bar graph to visualize top 10 genre most common genres of apps 
#fig 8
Genre_counts=app_df['Genres'].str.split(';',expand=True).stack().value_counts().nlargest(10)#it will acess genere column in the dataframe then str split split the genre splits stack sperates the genres in dofferent rows and then stacks it into on e column
fig8=px.bar(
    x=Genre_counts.index,#gives the value of x axis 
    y=Genre_counts.values,#gives the value of y  axis 
    labels={'x':'Genre','y':'Count'},#naming for x and y
    title='Top Genre', 
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.OrRd,
    width=400,
    height=300
)
#code for layout of the fig1 bar graph 
fig8.update_layout(  #updating the layout 
    plot_bgcolor='black',#ploting the backgroud color which is black  
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig8,"Genre Graph 8.html","Action and casual genres are the most common, reflecting users preferences for engaging and easy-to- play games")

In [176]:
#fig 9 here want to analyze the relationship btw the last update date and the app rating by scatter plot basically it explore weather the more keep on sending sending update on the app 
fig9=px.scatter(
    app_df,
    x='Last Updated',#gives the value of x axis 
    y='Rating',#gives the value of y  axis 
    color='Type',
    title='Impact on last update on Rating', 
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=400,
    height=300
)
#code for layout of the fig1 bar graph 
fig9.update_layout(  #updating the layout 
    plot_bgcolor='black',#ploting the backgroud color which is black  
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig9,"update Graph 9.html","The scatter plot a weak correlation between the last update and ratings suggesting that more frequent don't always resuls in better rating")

In [177]:
#create box plot to compare the rationgs of paid and free apps basically the box plot is graph to display a distribution, central tendency and variability 
#to identify outliers in my data understanding the spreading and skewness of the data 
#center box reprent IQR and contains middle 50% of the data, Bottom edge contains the 25% of the data and that the quartile 1 then the top edge is 3rd quartile thats a 75%
#the line is the median it represent the 50th percentile 
#then there wiskers which is coming extending from the box so basically from the smallest and the largest its 1.5times the interquartile range from the lower to upper quarter this will give the data range which does the include the outliers.
#outliers are data points which falls outside these wiskers so it will help to understand how much outliers is there in the data
#Box plot is used to compare distribution btw multiple groups   
#here we will be using the distribution of ratings and if there is any difference that is central tendency or the spread of the ratings  
fig10=px.box(
    app_df,
    x='Type',#gives the value of x axis 
    y='Rating',#gives the value of y  axis 
    color='Type',#based on the paid and free the color will be assigned 
    title='Rating for paid vs free Apps', 
    color_discrete_sequence=px.colors.qualitative.Pastel,
    width=400, 
    height=300
)
#code for layout of the fig1 bar graph 
fig10.update_layout(  #updating the layout 
    plot_bgcolor='black',#ploting the backgroud color which is black  
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
save_plot_as_html(fig10,"Paid free Graph 10.html","Paid apps genrally have higher ratings comparing to free apps, suggesting that user xpect higher quality from apps they pay for")

In [178]:
 #code in such a way that when plotly plots combines into single html file they have to be in there specific container 
 #1st splitting the plot containers to handle the last plot  
plot_containers_split=plot_containers.split('</div ')#here the plot containers is the string that is containg an html content of the multiple plotly plots 
#.split method basically splits the string into a list of substring whenever it encounters the /div it is going to split because each plotly lot ends with the div tag  
#so, splitting on this stack will allow us to seperate the different plots 

In [179]:
#now check if there have been multiple plots  
if len(plot_containers_split) > 1:
    final_plot=plot_containers_split[-2]+'</div>' #the final plot has to split againm if condition is true this line will construct the final plot content by taking the 2nd last part it will append the dev tag   
else:
    final_plot=plot_containers 
#to ensure last plotly plot is properly closed with div tag 
#now html code will be appropiate the plots is embeding in sigle html file they will not merge and overlape and be in there respective positions 

In [180]:
#now to create a web based dashboard using html and plotly  
#web base dashboard allows you to present your analysis visually in a web browser in an html page
#tkinter dashboard it is a python library to create front end of our python but not to use because it is not very advanced website when it comes to styling not that visually appealing
#because html css javascript provides a better way of presenting your data so we will be using html 
#CSS styling dark background with white text, header class container class, then plot container to add border,then insights, then jss functionality use to open a plotly graph in a new tab  
dashboard_html= """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset = "UTF-8">
    <meta name = viewport "content="width=device-width,initial-scale-1.0">
    <title> Google Playstore review Analytics</title>
    <style>
        body {{ 
            font-family: Arial, sans-serif;
            bacground-color: #333;
            color: #fff;
            margin: 0;
            padding: 0;
        }} 
        .header {{
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 20px;
            background-color: #444;
        }}
        .header img {{
            margin: 0 10px;
            height: 50px;
        }}
        .container{{
            display: flex;
            flex-wrap: wrap;
            justify_content: center;
            padding: 20px;
        }}
        .plot-container {{
            border: 2px solid #555
            margin: 10px;
            padding: 10px;
            width: {plot_width}px;
            height: {plot_height}px;
            overflow: hidden;
            position: relative;
            curson: pointer;
        }}
        .insights {{
            display: none;
            position: absolute;
            right: 10px
            top: 10px
            background-color: rgba(0,0,0,0.7);
            padding: 5px;
            border-radius: 5px;
            color: #fff;
        }}
        .plot-container: hover.insights  {{
            display: block;
        }}
        </style> 
        <script>
            function open plot(filename) {{
                window.open(filename, '_blank');
                }}
        </script>
</head>
<body>
     <div class= "header">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo">
        <h1>Google PLay Store Reviews Analytics </h1>
        <img src="file:///Users/prateeksingh/Downloads/Google_Play-Badge-Logo.wine.svg" alt="Google Play Store Logo">
     </div>
    <div class="container">
        {plots}
    </div>
</body>
</html> 
"""

In [181]:
#after html code now want to use the containers to fill my dashboards 
final_html=dashboard_html.format(plots=plot_containers,plot_width=plot_width,plot_height=plot_height)#  use to create the final html content for dashboard by fill the place holder with actual content 

In [182]:
dashboard_path=os.path.join(desktop_path," web page.html")#it will it as an html page in that particular path which i have given 

In [183]:
with open(dashboard_path,"w", encoding="utf-8") as f:
    f.write(final_html)

In [184]:
webbrowser.open('file://'+os.path.realpath(dashboard_path))

True